In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import input_file_name
from pyspark.sql.functions import explode

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
headers = ['categoria', 'titulo', 'link']
data = spark.read.option("recursiveFileLookup", "true").csv("s3://parcial2/headlines/final/").toDF(*headers)

data = data.filter(data["categoria"] != "categoria")

data = data.dropna(subset=["categoria","titulo","link"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
row_count = data.count()

data.show(row_count)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|           categoria|              titulo|                link|
+--------------------+--------------------+--------------------+
|                EEUU| Las impactantes ...| eltiempo.com/mun...|
|Conflicto y Narco...| Gobernador del M...| eltiempo.com/jus...|
|            Ciclismo| Rigoberto Urán a...| eltiempo.com/dep...|
|            Medellín| El macabro crime...| fue estrangulada...|
|Contenido Patroci...| ¿Por qué su empr...| eltiempo.com/con...|
|       Más Contenido| eCommerce Day Co...| el evento más im...|
|Contenido Patroci...| ¿Por qué su empr...| eltiempo.com/con...|
|       Más Contenido| eCommerce Day Co...| el evento más im...|
|               Gente| La modelo de Onl...| eltiempo.com/cul...|
|              Bogotá| Así es vivir en ...| el barrio más pe...|
|   Fútbol Colombiano| Los pecados que ...| eltiempo.com/dep...|
|            Medellín|      Cayó 'la coja'| acusada de robar...|
|           Venezuela| ¿P

In [41]:
data_with_filename = data.withColumn("filename", input_file_name())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
tokenizer = Tokenizer(inputCol="titulo", outputCol="palabras_titulo")
stopwords_remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="titulo_sin_stopwords")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
count_vectorizer = CountVectorizer(inputCol=stopwords_remover.getOutputCol(), outputCol="features")
idf = IDF(inputCol=count_vectorizer.getOutputCol(), outputCol="tfidf")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
label_indexer = StringIndexer(inputCol="categoria", outputCol="label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
pipeline = Pipeline(stages=[tokenizer, stopwords_remover, count_vectorizer, idf, label_indexer])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
model = pipeline.fit(data_with_filename)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
transformed_data = model.transform(data_with_filename)

transformed_data.write.mode("overwrite").parquet("s3://parcial2/headlines/final/datos_procesados.parquet")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
exploded_data = transformed_data.selectExpr("categoria", "titulo", "link")

exploded_data.write.mode("overwrite").csv("s3://parcial2/headlines/final/exploded_data.csv", header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…